In [335]:
# import sys
# !{sys.executable} -m pip install pytz
# !{sys.executable} -m pip install tzlocal
# !{sys.executable} -m pip install geopy

In [539]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime
import pytz
from tzlocal import get_localzone
from geopy import distance
from IPython.display import Markdown, Latex
from vending_token import load_vending_token

In [542]:
access_token = load_vending_token()
# access_token

In [339]:
def auth_headers(token = access_token):
    return { "Authorization": f"Bearer {token}" }

In [340]:
vending_url = "https://api.byu.edu/domains/vending/v1/vending.ashx"

In [341]:
category_list_params = {
    "format": "json",
    "service": "merchandise",
    "action": "listCategories"
}

category_list_response = requests.get(vending_url, headers = auth_headers(), params = category_list_params)

In [342]:
category_list = category_list_response.json()
category_list

[{'id': 1000, 'description': 'Food/Snacks'},
 {'id': 1001, 'description': 'Drinks'},
 {'id': 1002, 'description': 'Candy'},
 {'id': 1003, 'description': 'Ice Cream/Novelties'}]

In [343]:
target_category = 1000

In [344]:
product_list_params = {
    "format": "json",
    "service": "merchandise",
    "action": "getProducts",
    "cat": target_category
}

product_list_response = requests.get(vending_url, headers = auth_headers(), params = product_list_params)

In [345]:
product_list = product_list_response.json()
product_list[:5]

[{'id': 5243,
  'description': 'Sunbelt Choc. Chip Granola Bar',
  'price': '50',
  'img': ''},
 {'id': 5302, 'description': "Gardetto's", 'price': '80', 'img': ''},
 {'id': 5303,
  'description': 'Sun Chips Harvest Cheddar',
  'price': '100',
  'img': ''},
 {'id': 5304, 'description': 'Rold Gold Pretzels', 'price': '100', 'img': ''},
 {'id': 5306, 'description': 'Doritos Nacho Chips', 'price': '100', 'img': ''}]

In [346]:
sandwiches = [x for x in product_list if "sandwich" in x["description"].lower()]
sandwiches

[{'id': 5670,
  'description': 'Ritz Bitz Cheese Sandwich',
  'price': '90',
  'img': ''},
 {'id': 6114,
  'description': 'Italian Ciabatta Sandwich',
  'price': '350',
  'img': ''},
 {'id': 6150,
  'description': 'Tuna Salad 1/2 Sandwich',
  'price': '200',
  'img': ''},
 {'id': 6463,
  'description': 'MM Ritz Bitz Cracker Sandwich',
  'price': '100',
  'img': ''},
 {'id': 6677,
  'description': 'MM Tuna  Wedge Sandwich',
  'price': '250',
  'img': ''},
 {'id': 6879,
  'description': 'MM Sandwich Cremes Duplex',
  'price': '125',
  'img': ''}]

In [347]:
byu_sandwich_ids = [
    6114, #Italian Ciabatta
    # And I think that's it? Where are the others?
]

In [373]:
croissants = [x for x in product_list if "croissant" in x["description"].lower()]
croissants

[{'id': 5985,
  'description': 'Chicken Salad Croissant',
  'price': '350',
  'img': ''},
 {'id': 6050, 'description': 'Ham Croissant', 'price': '350', 'img': ''},
 {'id': 6476,
  'description': 'Croissant Sausage Egg and Cheese',
  'price': '350',
  'img': ''},
 {'id': 6592,
  'description': 'Turkey Gouda Croissant',
  'price': '350',
  'img': ''}]

In [374]:
byu_croissant_ids = [
    5985, # Chicken Salad
    6592, # Turkey Gouda
    6050, # Ham & Swiss
]

In [375]:
bagels = [x for x in product_list if "bagel" in x["description"].lower()]
bagels

[{'id': 5977,
  'description': 'Blueberry BAGELw/ Cream Cheese',
  'price': '125',
  'img': ''},
 {'id': 5978,
  'description': 'Plain Bagel w/ Cream Cheese',
  'price': '125',
  'img': ''},
 {'id': 5996, 'description': 'Pizza Bagel', 'price': '125', 'img': ''},
 {'id': 5997, 'description': 'Asiago Bagel', 'price': '125', 'img': ''}]

In [376]:
byu_bagel_ids = [
    5997, # Asiago
    5977, # Blueberry
    5978, # Plain
    5996, # Pizza
]

Nutritional reference:
- sandwich(es??) https://dining.byu.edu/vending/images/nutrition/nutrition_Sandwiches.png
- croissants https://dining.byu.edu/vending/images/nutrition/nutrition_Croissants.png
- bagels (except pizza bagel?) https://dining.byu.edu/vending/images/nutrition/nutrition_Bagels.png

In [377]:
nutrition_data = pd.read_csv('./bakery_nutrition.csv')
nutrition_data[:3]

,id,cost,calories,protein,carbohydrates,sodium,fiber,sugar,fat
0,6114,350,645,31,57,1903,2,5,34.0
1,5985,350,772,24,92,1061,6,36,46.0
2,6592,350,857,33,67,1862,2,9,60.0


Taken from [FDA daily recommended values document](https://www.fda.gov/media/99059/download) for ages >=4:

In [474]:
drv_data = pd.read_csv('./fda_drv.csv')
drv_data

,name,unit,quantity
0,fat,g,78
1,saturated_fat,g,20
2,cholesterol,mg,300
3,carbohydrates,g,275
4,sodium,mg,2300
5,fiber,g,28
6,protein,g,50
7,sugar,g,50


In [483]:
drv_data.loc[drv_data['name'] == 'sugar']

,name,unit,quantity
7,sugar,g,50


In [488]:
nutrition_data['name'] = nutrition_data.apply(lambda row: next(x['description'] for x in product_list if x["id"] == int(row.id)), axis = 1)
nutrition_data['calories_dollar'] = nutrition_data.apply(lambda row: row.calories / (row.cost / 100), axis = 1)
nutrition_data['protein_dollar'] = nutrition_data.apply(lambda row: row.protein / (row.cost / 100), axis = 1)
nutrition_data['fiber_dollar'] = nutrition_data.apply(lambda row: row.fiber / (row.cost / 100), axis = 1)
nutrition_data['protein_drv'] = nutrition_data.apply(lambda row: row.protein / drv_data.loc[drv_data['name'] == 'protein'].quantity, axis = 1)
nutrition_data['fiber_drv'] = nutrition_data.apply(lambda row: row.fiber / drv_data.loc[drv_data['name'] == 'fiber'].quantity, axis = 1)
nutrition_data[:3]

,id,cost,calories,protein,carbohydrates,sodium,fiber,sugar,fat,name,dollar_calories,dollar_protein,dollar_fiber,calories_dollar,protein_dollar,fiber_dollar,protein_drv,fiber_drv
0,6114,350,645,31,57,1903,2,5,34.0,Italian Ciabatta Sandwich,184.285714,8.857143,0.571429,184.285714,8.857143,0.571429,0.62,0.071429
1,5985,350,772,24,92,1061,6,36,46.0,Chicken Salad Croissant,220.571429,6.857143,1.714286,220.571429,6.857143,1.714286,0.48,0.214286
2,6592,350,857,33,67,1862,2,9,60.0,Turkey Gouda Croissant,244.857143,9.428571,0.571429,244.857143,9.428571,0.571429,0.66,0.071429


In [461]:
highest_calories_per_dollar = nutrition_data.sort_values(by=['dollar_calories'], ascending=False)
highest_calories_per_dollar[:5]

,id,cost,calories,protein,carbohydrates,sodium,fiber,sugar,fat,name,dollar_calories,dollar_protein,dollar_fiber
5,5997,125,316,13,60,602,3,0,2.5,Asiago Bagel,252.800000,10.400000,2.400000
6,5977,125,315,11,67,545,3,11,0.0,Blueberry BAGELw/ Cream Cheese,252.000000,8.800000,2.400000
2,6592,350,857,33,67,1862,2,9,60.0,Turkey Gouda Croissant,244.857143,9.428571,0.571429
7,5978,125,289,12,60,531,3,0,0.0,Plain Bagel w/ Cream Cheese,231.200000,9.600000,2.400000
4,6146,350,790,17,133,1201,16,19,37.0,Wrap Turmeric Curry Garbanzo,225.714286,4.857143,4.571429


In [462]:
highest_protein_per_dollar = nutrition_data.sort_values(by=['dollar_protein'], ascending=False)
highest_protein_per_dollar[:5]

,id,cost,calories,protein,carbohydrates,sodium,fiber,sugar,fat,name,dollar_calories,dollar_protein,dollar_fiber
5,5997,125,316,13,60,602,3,0,2.5,Asiago Bagel,252.800000,10.400000,2.400000
7,5978,125,289,12,60,531,3,0,0.0,Plain Bagel w/ Cream Cheese,231.200000,9.600000,2.400000
2,6592,350,857,33,67,1862,2,9,60.0,Turkey Gouda Croissant,244.857143,9.428571,0.571429
0,6114,350,645,31,57,1903,2,5,34.0,Italian Ciabatta Sandwich,184.285714,8.857143,0.571429
6,5977,125,315,11,67,545,3,11,0.0,Blueberry BAGELw/ Cream Cheese,252.000000,8.800000,2.400000


In [522]:
search_term = "gouda"

In [523]:
target_product = next(x for x in product_list if search_term.lower() in x["description"].lower())
target_product

{'id': 6592,
 'description': 'Turkey Gouda Croissant',
 'price': '350',
 'img': ''}

In [524]:
target_product_id = target_product['id']
target_product_id

6592

In [525]:
sandwiches

[{'id': 5670,
  'description': 'Ritz Bitz Cheese Sandwich',
  'price': '90',
  'img': ''},
 {'id': 6114,
  'description': 'Italian Ciabatta Sandwich',
  'price': '350',
  'img': ''},
 {'id': 6150,
  'description': 'Tuna Salad 1/2 Sandwich',
  'price': '200',
  'img': ''},
 {'id': 6463,
  'description': 'MM Ritz Bitz Cracker Sandwich',
  'price': '100',
  'img': ''},
 {'id': 6677,
  'description': 'MM Tuna  Wedge Sandwich',
  'price': '250',
  'img': ''},
 {'id': 6879,
  'description': 'MM Sandwich Cremes Duplex',
  'price': '125',
  'img': ''}]

This is where it will help to have your accurate location:

In [526]:
ex_lat_long = [40.249304, -111.651168]

In [527]:
in_stock_params = {
    "format": "json",
    "service": "inventory",
    "action": "findMachinesWithStock",
    "product": target_product_id
}

in_stock_response = requests.get(vending_url, headers = auth_headers(), params = in_stock_params)

In [528]:
in_stock_list = in_stock_response.json()
in_stock_list[:2]

[{'location': '5010',
  'description': '101 BRWB',
  'lat': '40.24673',
  'lng': '-111.64532',
  'items': [{'updateTime': '2020-10-0115:40:00.0-06:00',
    'price': '350',
    'amount': 3,
    'item': 'Turkey Gouda Croissant',
    'img': ''}]},
 {'location': '5011',
  'description': 'BRMB 144',
  'lat': '40.24632',
  'lng': '-111.65241',
  'items': [{'updateTime': '2020-10-0109:06:00.0-06:00',
    'price': '350',
    'amount': 0,
    'item': 'Turkey Gouda Croissant',
    'img': ''}]}]

In [529]:
def item_dist(lat_lng, item):
    item_lat_lng = list(map(float, [item["lat"], item["lng"]]))
    return distance.distance(np.array(item_lat_lng), np.array(lat_lng))

In [530]:
def add_distances_to_item(lat_lng, item):
    dist = item_dist(lat_lng, item)
    return {**item, "km": dist.km, "ft": dist.ft}

In [531]:
closest_items = list(map(lambda stock: add_distances_to_item(ex_lat_long, stock), in_stock_list))
# closest_items[:2]

In [532]:
closest_items_sorted = sorted(closest_items, key=lambda stock: stock["km"])
# closest_items_sorted[:2]

In [533]:
def pretty_vending_time(time_str):
    before_dot = time_str.split('.')[0]
    time_str = f"{before_dot[:10]}T{before_dot[10:]}-{time_str.split('-')[-1]}"
    local_tz = get_localzone()
    return datetime.fromisoformat(time_str).astimezone(local_tz).strftime("%x %X")

In [534]:
closest_stock_item = closest_items_sorted[0]
closest_stock_item

{'location': '5090',
 'description': 'TMCB 121',
 'lat': '40.24931',
 'lng': '-111.65058',
 'items': [{'updateTime': '2020-10-0108:57:00.0-06:00',
   'price': '350',
   'amount': 2,
   'item': 'Turkey Gouda Croissant',
   'img': ''}],
 'km': 0.05003294100493905,
 'ft': 164.150068913842}

In [535]:
closest_stock_item_details = closest_stock_item["items"][0]
closest_stock_item_details

{'updateTime': '2020-10-0108:57:00.0-06:00',
 'price': '350',
 'amount': 2,
 'item': 'Turkey Gouda Croissant',
 'img': ''}

In [536]:
closest_stock_item_update_time_str = pretty_vending_time(closest_stock_item_details["updateTime"])
closest_stock_item_update_time_str

'10/01/20 08:57:00'

In [537]:
closest_nutrition_info = nutrition_data.loc[nutrition_data['id'] == target_product_id]
if not closest_nutrition_info.empty:
    closest_calories = int(closest_nutrition_info['calories'])
    closest_protein = int(closest_nutrition_info['protein'])
    closest_fiber = int(closest_nutrition_info['fiber'])
    closest_protein_drv = '{0:.02f}'.format(round(float(closest_nutrition_info['protein_drv']) * 100, 2))
    closest_fiber_drv = '{0:.02f}'.format(round(float(closest_nutrition_info['fiber_drv']) * 100, 2))
    closest_calories_per_dollar = round(float(closest_nutrition_info['calories_dollar']), 2)
    closest_protein_per_dollar = round(float(closest_nutrition_info['protein_dollar']), 2)
    closest_fiber_per_dollar = round(float(closest_nutrition_info['fiber_dollar']), 2)

In [538]:
closest_name = closest_stock_item_details['item']
closest_ft_dist = round(closest_stock_item['ft'])
closest_location_name = closest_stock_item['description']
closest_quantity = closest_stock_item_details['amount']
closest_price = '{0:.02f}'.format(int(closest_stock_item_details['price']) / 100)

nutrition_str = ""

if not closest_nutrition_info.empty:
    nutrition_str = f"""
Contains:
- {closest_calories} calories
- {closest_protein}g protein ({closest_protein_drv}% daily recommended value)
- {closest_fiber}g fiber ({closest_fiber_drv}% daily recommended value)
- {closest_calories_per_dollar} calories/dollar
- {closest_protein_per_dollar}g protein/dollar
- {closest_fiber_per_dollar}g fiber/dollar
"""

info_str = f"""
### The closest {closest_name} at BYU is {closest_ft_dist} ft away, at {closest_location_name}, for ${closest_price}.

{nutrition_str}

At __{closest_stock_item_update_time_str}__, there {"were" if closest_quantity != 1 else "was"}
__{closest_quantity}__ available.
"""

display(Markdown(info_str))


### The closest Turkey Gouda Croissant at BYU is 164 ft away, at TMCB 121, for $3.50.


Contains:
- 857 calories
- 33g protein (66.00% daily recommended value)
- 2g fiber (7.14% daily recommended value)
- 244.86 calories/dollar
- 9.43g protein/dollar
- 0.57g fiber/dollar


At __10/01/20 08:57:00__, there were
__2__ available.
